### The following code establishes connection with snowflake using snowpark.
### Note: snowpark uses 3.8.16
The code then:

1. executes sql code in python
2. works with snowpark dataframe to call and merge snowflake tables
3. publishes snowpark dataframe to snowflake

In [ ]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from snowflake.snowpark import Session
import pandas as pd

In [ ]:
connection_parameters = {
    "user": "user", #update for your own account
    "authenticator": "externalbrowser",
    "account": "account", #update for your own account
    "database": "database", #update for your own account
    "schema": "schema",} #update for your own account
 

session = Session.builder.configs(connection_parameters).create()

Quering data 
1. ..Will not query data for below on snowflake <br> query_1 = session.sql("code")
2. .. Will query data fro below on snowflake because of **collect()** <br> query_2 = session.sql("code").collect()
3. .. Will query data for query_1 because **show()** <br> query_1.show()

In [ ]:
q = """
    create or replace temp table "temp_table1" as( 
    SELECT distinct col1,
           col2,
           col3 AS col_alias1,
           1 AS col_alias2
    FROM schema1.table1 col_alias3 
    WHERE col4 = 'VALUE'
        AND col1 IS NOT NULL 
        AND YEAR(col5) = [YEAR]
        AND MONTH(col5) <= [MONTH])
"""

# 1.
session.sql(q)

# 2.
session.sql(q).collect()

# 3.
query_2 = session.sql(q).show()

In [ ]:
# You can also open a sql text file and execute separate queries
# Open the sql text file in read mode
generic_file_path = "./generic_file.sql"
text_file = open(generic_file_path, "r")

# Read the whole file to a string
generic_sql_content = text_file.read()

# Close the file
text_file.close()

# Execute the content from the file
generic_result1 = session.sql(generic_sql_content.split(';')[0]).collect()
generic_result2 = session.sql(generic_sql_content.split(';')[1]).collect()
generic_result3 = session.sql(generic_sql_content.split(';')[2]).collect()

# Read in generic tables - use to check that execution ran properly
df_table1 = session.table("generic_schema.generic_table1")
df_table2 = session.table("generic_schema.generic_table2")
df_table3 = session.table("generic_schema.generic_table3")

df_table1.show()
df_table2.show()
df_table3.show()


#### Creating Snowpark Dataframe by directly reading a table 
<br> df_tbl_ad = session.table("code") 
<br> #df_tbl_read.show()

In [ ]:
df = session.table("analytics.cacp_y1_ergo").show()

Creating Snowpark Dataframe by joining two tables/dataframes
<br> df_tbl2_read = session.table("code")
<br> df_join = df_tbl_read.join(df_tbl2_read, df_tbl_read["var"] == df_tbl2_read["var"])
<br> df_join.show()

In [ ]:
df_generic1_read = session.table("generic_schema.table1")
df_generic2_read = session.table("generic_schema.table2")
df_join_generic = df_generic1_read.join(df_generic2_read, df_generic1_read["generic_col1"] == df_generic2_read["generic_col2"])
df_join.show()

Publishing output

In [ ]:
# generic_function_name function
generic_df = generic_function_name(generic_query, generic_list)

# export to a generic Excel file
with pd.ExcelWriter('generic_filename.xlsx') as writer:
    generic_df.to_excel(writer, sheet_name="generic_sheet_name", index=False)

# write to a generic database
df.write.mode("overwrite").saveAsTable("generic_schema.generic_table_name")
